# Hello, _nbpresent_!

In [8]:
import nbpresent
nbpresent.__version__

'3.0.2'



### You're going to need the following things
- A working instance of Python, complete with the following libraries
  - comtypes
  - pandas
  - {some other stuff}

First we import references to comtypes, and specifically the CreateObject method


In [9]:
from win32api import GetSystemMetrics
import comtypes
from comtypes.client import CreateObject

set up the STK (AgUiApplication)

In [10]:
app=CreateObject("STK11.Application")

In [11]:
app.Visible=True
app.UserControl=True

In [12]:
app.Top=0
app.Left=0
app.Width=int(GetSystemMetrics(0)/2)
app.Height=int(GetSystemMetrics(1)-30)

In [ ]:
root=app.Personality2

In [19]:
type(root)

comtypes.POINTER(_IAgStkObjectRoot)

Next, we're going to assign two new variables to the autogenerated Python version of the STK Classes found in `comtypes.gen.STKObjects` and `comtypes.gen.STKUtil`.  These will be useful when transitioning between Interfaces exposed by each STK object we will use, as well as all the readable Enumerations (ex. eScenario = 19)

In [26]:
from comtypes.gen import STKUtil
from comtypes.gen import STKObjects

In [37]:
root.NewScenario("IPython_DIY")

0

In [58]:
_i38

'app.Top=0\napp.Left=0\napp.Width=int(GetSystemMetrics(0)/2)\napp.Height=int(GetSystemMetrics(1)-30)'

In [55]:
from IPython.display import IFrame
IFrame("http://help.agi.com/stkdevkit/index.html", width=700, height=350)

In [40]:
sc=root.CurrentScenario
sc

<POINTER(IAgStkObject) ptr=0x12be1a98 at 8dbd7c8>

The object interface we get back is of type **_IAgStkObject_** but we need to work with **_IAgScenario_** properties and methods such as `SetTimePeriod()`, so we need to "cast" the sc object into it's **_IAgScenario_** Interface.  We do that via the `QueryInterface`() generic method on the sc object and simply set the sc object to that change. We use the `STKObjects.IAgScenario` "knowledge" gained when the STK Type Libraries were scanned and those equivalent Python Classes created.

In [41]:
sc=sc.QueryInterface(STKObjects.IAgScenario)
sc

<POINTER(IAgScenario) ptr=0x11dcbc28 at 7671f48>

In [43]:
sc.SetTimePeriod("10 Jun 2016 04:00:00","11 Jun 2016 04:00:00")
root.Rewind()

0

In [44]:
sc=sc.QueryInterface(STKObjects.IAgStkObject)
GSFC= sc.Children.New(STKObjects.eFacility,"GSFC")

GSFC = GSFC.QueryInterface(STKObjects.IAgFacility)
GSFC.Position.AssignGeodetic(38.9943,-76.8489,0)

0

In [49]:
sat = sc.Children.New(STKObjects.eSatellite, "codeSat")
sat= sat.QueryInterface(STKObjects.IAgSatellite)
sat.SetPropagatorType(STKObjects.ePropagatorJ2Perturbation)
 
satProp = sat.Propagator
satProp=satProp.QueryInterface(STKObjects.IAgVePropagatorJ2Perturbation)
satProp.InitialState.Epoch="08 Jun 2016 15:14:26"

keplerian = satProp.InitialState.Representation.ConvertTo(STKUtil.eOrbitStateClassical)
keplerian = keplerian.QueryInterface(STKObjects.IAgOrbitStateClassical)
keplerian.SizeShapeType =STKObjects.eSizeShapeMeanMotion
keplerian.LocationType = STKObjects.eLocationMeanAnomaly
keplerian.Orientation.AscNodeType = STKObjects.eAscNodeRAAN

root.UnitPreferences.Item('Angle').SetCurrentUnit('revs')
root.UnitPreferences.Item('Time').SetCurrentUnit('day')

keplerian.SizeShape.QueryInterface(STKObjects.IAgClassicalSizeShapeMeanMotion).MeanMotion = 15.08385840
keplerian.SizeShape.QueryInterface(STKObjects.IAgClassicalSizeShapeMeanMotion).Eccentricity = 0.0002947

root.UnitPreferences.Item('Angle').SetCurrentUnit('deg')
root.UnitPreferences.Item('Time').SetCurrentUnit('sec')

keplerian.Orientation.Inclination = 2.84703
keplerian.Orientation.ArgOfPerigee = 114.7239
keplerian.Orientation.AscNode.QueryInterface(STKObjects.IAgOrientationAscNodeRAAN).Value = 315.1965
keplerian.Location.QueryInterface(STKObjects.IAgClassicalLocationMeanAnomaly).Value = 332.9096

satProp.InitialState.Representation.Assign(keplerian)
satProp.Propagate()

0